In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110520

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,73894.5978
2018-02-28,85534.8055
2018-03-31,64702.9059
2018-04-30,62590.7426
2018-05-31,72019.8542
2018-06-30,67292.0623
2018-07-31,86516.3448
2018-08-31,65257.6890
2018-09-30,69828.9550


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31     73894.5978
 2018-02-28     85534.8055
 2018-03-31     64702.9059
 2018-04-30     62590.7426
 2018-05-31     72019.8542
 2018-06-30     67292.0623
 2018-07-31     86516.3448
 2018-08-31     65257.6890
 2018-09-30     69828.9550
 2018-10-31     57119.9481
 2018-11-30     68674.8951
 2018-12-31     82188.4312
 2019-01-31    106169.9080
 2019-02-28     78670.2180
 2019-03-31     86716.5718
 2019-04-30     87604.5541
 2019-05-31     88686.3352
 2019-06-30     72029.9837
 2019-07-31    102419.8955
 2019-08-31     93911.5011
 2019-09-30     96097.2991
 2019-10-31     81800.4168
 2019-11-30     95456.3255
 2019-12-31    102546.0727
 2020-01-31    108502.8644
 2020-02-29     92568.6306
 2020-03-31     84822.7931
 2020-04-30     69326.0599
 2020-05-31     77612.6154
 2020-06-30     75129.3594
 2020-07-31     94487.2199
 2020-08-31     87171.3494
 2020-09-30     89813.9656
 2020-10-31     82487.5710
 2020-11-30     75940.4364
 2020-12-31     83367.0806

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -1.357974
p-value : 0.602250
Critical Values :
	1%: -3.632743
	5%: -2.948510
	10%: -2.613017


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.918482
p-value : 0.000032
Critical Values :
	1%: -3.626652
	5%: -2.945951
	10%: -2.611671


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=911.419, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=907.898, Time=0.11 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=904.222, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=907.364, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=910.570, Time=0.11 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=916.487, Time=0.14 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=904.364, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=905.929, Time=0.06 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=906.396, Time=0.11 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=908.023, Time=0.14 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=910.089, Time=0.14 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=905.002, Time=0.06 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=903.843, Time=0.16 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=905.511, Time=0.25 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=906.000, Time=0.21 sec
 ARIMA(3,0,0)(0,0,0)[1] i

ARIMA(order=(2, 0, 1), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(2,0,1))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(2, 1)   Log Likelihood                -446.230
Method:                       css-mle   S.D. of innovations           9872.010
Date:                Sat, 11 Sep 2021   AIC                            902.460
Time:                        04:00:46   BIC                            911.149
Sample:                             0   HQIC                           905.645
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        8.16e+04   3228.302     25.277      0.000    7.53e+04    8.79e+04
ar.L1.y       -0.3894      0.157     -2.479      0.013      -0.697      -0.082
ar.L2.y        0.5404      0.126      4.300      0.0

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([80042.40638573, 79387.23405698]), array([ 9872.01010925, 10867.92371778]), array([[ 60693.62211659,  99391.19065487],
       [ 58086.4949834 , 100687.97313056]]))
